# Evaluating RAG (Actually Reverser-Reverse RAG)

In [1]:
import os
import json
import dspy
import pandas as pd


d:\ThirdSummer\Something\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dspy import ColBERTv2

llama = dspy.GROQ(model="llama3-8b-8192", api_key=os.environ["GROQ_API_KEY"])
turbo = dspy.OpenAI(model="gpt-3.5-turbo", api_key=os.environ["OPENAI_API_KEY"])
mini = dspy.OpenAI(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])

colber_wiki = ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")

dspy.settings.configure(lm=mini, rm=colber_wiki)

In [4]:
trivia_df = pd.read_csv("./input_data/pollution_trivia_gpt4omini.csv")
trivia_df = trivia_df[['question', 'answer']]
trivia_df.head()

,question,answer
0,What is the primary organ affected by lead poi...,Brain
1,What is the primary condition caused by lead p...,Avian plumbism
2,What type of pollution is commonly associated ...,Water contamination
3,What is the main environmental concern with hy...,Water contamination
4,What is the name of the famous oil spill that ...,Exxon Valdez


In [5]:
train_set = trivia_df.iloc[:-35].to_dict(orient="records")
test_set = trivia_df.iloc[-35:].to_dict(orient="records")

print(len(train_set), len(test_set))

train_set = [dspy.Example(question=ex["question"], answer=ex["answer"]).with_inputs('question') for ex in train_set]
test_set = [dspy.Example(question=ex["question"], answer=ex["answer"]).with_inputs('question') for ex in test_set]



81 35


In [6]:
from dsp.utils import deduplicate
from pydantic import BaseModel

class GenerateAnswer(dspy.Signature):
    """ Answer question with short factual answer """
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")
    
class GenrerateSearchQuestion(dspy.Signature):
    """ Write a simple search question that will help answer a complex query """
    question = dspy.InputField()
    query = dspy.OutputField(desc="Name of a concept, entity, event or topic that can be used to search for more information")
    
class RAG(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_query = dspy.Predict(GenrerateSearchQuestion)
        self.retrieve = dspy.Retrieve(k=2)
        self.generate_answer = dspy.Predict(GenerateAnswer)
        
    def forward(self, question):
        query = self.generate_query(question=question).query
        context = self.retrieve(query).passages
        answer = self.generate_answer(question=question, context=context).answer
        return dspy.Prediction(
            query=query, 
            context=context, 
            answer=answer
        )

    


In [7]:
class JudgeQA(dspy.Signature):
    """ Given the question, determine if the two answers below mean the same thing """
    question = dspy.InputField()
    answer1 = dspy.InputField()
    answer2 = dspy.InputField()
    is_same = dspy.OutputField(desc="Yes or No")

import random
def t():
    return dict(temperature=0.7 + 0.0001 * random.uniform(-1, 1))

judge = dspy.ChainOfThought(JudgeQA, **t()) # tweaking temperature to avoid caching and can try different llms

def metric(example, pred, trace = None):
    return judge(question=example.question, answer1=example.answer, answer2=pred.answer).is_same.lower() == 'yes'


## How does llama perform on the trivia quiz

In [48]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(
    metric = metric,
    devset = test_set,
    display_progress = True,
    display_table = 10,
    return_outputs=True,
)
eval = evaluate(RAG())
print(eval)

ERROR:dspy.evaluate.evaluate:2024-08-12T14:33:41.184618Z [error    ] Error for example in dev set: 		 Connection error. [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


ERROR:dspy.evaluate.evaluate:2024-08-12T14:33:44.009086Z [error    ] Error for example in dev set: 		 Connection error. [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


ERROR:dspy.evaluate.evaluate:2024-08-12T14:33:46.983327Z [error    ] Error for example in dev set: 		 Connection error. [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


ERROR:dspy.evaluate.evaluate:2024-08-12T14:33:49.942664Z [error    ] Error for example in dev set: 		 Connection error. [dspy.evaluate.evaluate] filename=evaluate.py lineno=180


Average Metric: 18.0 / 32  (56.2):  91%|█████████▏| 32/35 [03:22<00:31, 10.59s/it]

APIConnectionError: Connection error.

## How does gpt perform on the trivia quiz

In [ ]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(
    metric = metric,
    devset = test_set,
    display_progress = True,
    display_table = 10,
    return_outputs=True,
)
eval = evaluate(RAG())
print(eval)

### Results; Llama: 40.0, Mini: 42.9


## Evaluating on T5 Model (without fine tuning)

In [53]:
import dspy 
from dspy import ColBERTv2

t5 = dspy.HFModel(model="google-t5/t5-base")
colber_wiki = ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")

dspy.settings.configure(lm=t5, rm=colber_wiki)

Average Metric: 18.0 / 32  (56.2):  91%|█████████▏| 32/35 [1:15:01<07:02, 140.68s/it]


In [54]:
from rich import print as rprint

t5_rag = RAG()
query = "What is the herbicide that endocrine disruptor, according to Tyrone Hayes' research?"
actual_answer = "Atrazine"

answer = t5_rag(question=query)

rprint("Question: ", query)
rprint("Actual Answer: ", actual_answer)
print()
rprint("Input Qesry: ", answer.query)
rprint("Context", answer.context[0])
rprint("Predicted Answer: ", answer.answer)

Question:  What is the herbicide that endocrine disruptor, according to Tyrone Hayes' research?

Actual Answer:  Atrazine

Input Qesry:  True

Context True (cigarette) | True is a brand of cigarettes marketed by R.J. Reynolds Tobacco Company.

Predicted Answer:  False

### Pretty Bad huh

## Fine Tuning T5-base with the outputs of GPT 4o Mini

In [8]:
from dspy.teleprompt import  BootstrapFinetune

tp = BootstrapFinetune(metric=None)

unlabeled_train_data = [dspy.Example(question=x["question"]).with_inputs('question') for x in train_set]
gpt_rag = RAG()

t5_base = dspy.HFModel(model="google-t5/t5-base")
t5_base.device = 'mps'

tf_rag = RAG()
for p in tf_rag.predictors():
    p.lm = t5_base

configurations = {
    "target": 't5-base',
    "epochs": 10,
    "bsize": 8,
    "accumsteps": 2,
    "lr": 5e-5,
}

rag_finetuned = tp.compile(
    student = tf_rag,
    teacher = gpt_rag,
    trainset = unlabeled_train_data,
    **configurations,
)

d:\ThirdSummer\Something\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 81/81 [00:00<00:00, 110.24it/s]


Bootstrapped 81 full traces after 81 examples in round 0.
all 162
local_cache/compiler\all.0d05e675e6e2c3c0.jsonl


Map: 100%|██████████| 162/162 [00:00<00:00, 5225.78 examples/s]


# examples skipped due to parsing error: 0 / 162


Map: 100%|██████████| 162/162 [00:00<00:00, 3097.69 examples/s]


...

{'eval_loss': 1.7605310678482056, 'eval_rouge1': 25.8964, 'eval_rouge2': 11.3971, 'eval_rougeL': 25.2991, 'eval_rougeLsum': 24.7342, 'eval_gen_len': 8.117647058823529, 'eval_runtime': 12.9989, 'eval_samples_per_second': 1.308, 'eval_steps_per_second': 0.231, 'epoch': 9.47}


100%|██████████| 3/3 [00:06<00:00,  1.99s/it]


{'train_runtime': 3341.4886, 'train_samples_per_second': 0.434, 'train_steps_per_second': 0.027, 'train_loss': 1.481752692328559, 'epoch': 9.47}


100%|██████████| 90/90 [55:41<00:00, 37.13s/it]


Best checkpoint of model: ../finetuning_ckpts\V8E8WA6VTUQTT.all\checkpoint-90
#> Best checkpoint path: ../finetuning_ckpts\V8E8WA6VTUQTT.all\checkpoint-90 for all
Assigning the LM of predictor all.
Assigning the LM of predictor all.


In [9]:
from rich import print as rprint

query = "What is the herbicide that endocrine disruptor, according to Tyrone Hayes' research?"
actual_answer = "Atrazine"

answer = rag_finetuned(question=query)

rprint("Question: ", query)
rprint("Actual Answer: ", actual_answer)
print()
rprint("Input Qesry: ", answer.query)
rprint("Context", answer.context[0])
rprint("Predicted Answer: ", answer.answer)

Question:  What is the herbicide that endocrine disruptor, according to Tyrone Hayes' research?

Actual Answer:  Atrazine

Input Qesry:  Name of a concept, entity, event or topic that can be used to search for more information: Herbicide 
that endocrine disruptor

Context Tyrone Hayes | Tyrone B. Hayes (born July 29, 1967) is an American biologist and professor of Integrative 
Biology at University of California, Berkeley known for his research findings concluding that the herbicide 
atrazine is an endocrine disruptor that demasculinizes and feminizes male frogs. He is also an advocate for 
critical review and regulation of pesticides and other chemicals that may cause adverse health effects. He has 
presented hundreds of papers, talks, and seminars on his conclusions that environmental chemical contaminants have 
played a role in global amphibian declines and in the health disparities that occur in minority and low income 
populations. His work has been contested by Syngenta, the Swiss manufacturer of atrazine and the Australian 
Pesticides and Veterinary Medicines Authority. It was used as the basis for the settlement of a multimillion-dollar
class-action lawsuit against Syngenta.

Predicted Answer:  Answer: Benzodiazepines

### Not that Dumb or is it???